# Mẫu Cơ Bản AutoGen

Trong ví dụ mã này, bạn sẽ sử dụng [AutoGen](https://aka.ms/ai-agents/autogen) AI Framework để tạo một agent cơ bản.

Mục tiêu của ví dụ này là để cho bạn thấy các bước mà chúng ta sẽ sử dụng sau này trong các ví dụ mã bổ sung khi triển khai các mẫu hành vi agent khác nhau.


## Nhập các Gói Python Cần Thiết


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Tạo Client

Trong ví dụ này, chúng ta sẽ sử dụng [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) để truy cập vào LLM.

`model` được định nghĩa là `gpt-4o-mini`. Hãy thử thay đổi model sang một model khác có sẵn trên marketplace của GitHub Models để xem các kết quả khác nhau.

Để kiểm tra nhanh, chúng ta sẽ chạy một prompt đơn giản - `Thủ đô của Pháp là gì`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Định nghĩa Agent

Bây giờ, sau khi chúng ta đã thiết lập `client` và xác nhận rằng nó hoạt động, hãy tạo một `AssistantAgent`. Mỗi agent có thể được gán:  
**name** - Một tên ngắn gọn, hữu ích để tham chiếu trong các luồng đa agent.  
**model_client** - Client mà bạn đã tạo ở bước trước.  
**tools** - Các công cụ có sẵn mà Agent có thể sử dụng để hoàn thành một nhiệm vụ.  
**system_message** - Metaprompt định nghĩa nhiệm vụ, hành vi và giọng điệu của LLM.  

Bạn có thể thay đổi system message để xem LLM phản hồi như thế nào. Chúng ta sẽ tìm hiểu về `tools` trong Bài học #4.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Chạy Agent

Hàm dưới đây sẽ chạy Agent. Chúng ta sử dụng phương thức `on_message` để cập nhật trạng thái của Agent với tin nhắn mới.

Trong trường hợp này, chúng ta cập nhật trạng thái với một tin nhắn mới từ người dùng là `"Plan me a great sunny vacation"`.

Bạn có thể thay đổi nội dung tin nhắn để xem cách LLM phản hồi khác nhau.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc sự không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn tham khảo chính thức. Đối với các thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp từ con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
